<a href="https://colab.research.google.com/github/elainechan01/hfrl-self-study/blob/main/unit1/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install dependencies**


*   gymnasium: Contains the LunarLander-v2 environment
*   stable-baselines3: The deep reinforcement learning library
*   hugging_face_sb3: Additi\onal code for Stable-baselines3 to load and upload models from the Hugging Face Hub



In [3]:
!apt install swig cmake

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,299 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubu

In [4]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 24.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of gymnasium[box2d] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 62.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none

In [5]:
!sudo apt-get update                      # virtual screen
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [973 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [55.6 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,126 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,994 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,258 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,410 kB]
Hit:13 https

In [ ]:
# restart notebook to ensure dependencies were properly installed
import os

os.kill(os.getpid(), 9)

In [2]:
# start virtual screen
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [3]:
# import statements

import gymnasium as gym

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import (
    notebook_login,
)  # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

**Set up Stable-Baselines3**

1. Create environment
2. Define model "MlpPolicy"
3. Train the agent



In [18]:
env = make_vec_env(lambda: gym.make('LunarLander-v2'), n_envs=4)   # make_vec_env creates multiple environments for parallel training

In [22]:
model = PPO(
    policy="MlpPolicy",
    env=env,
    batch_size=64,      # minibatch size
    n_steps=1024,       # number of steps to run for each env per update (horizon)
    n_epochs=10,        # number of epoch when optimizing the surrogate loss (more epochs can lead to better convergence but can also be slower)
    learning_rate=5e-4, # (a lower learning rate can help with stability)
    gamma=0.999,        # discount factor (< gamma, > discount ~= short term rewards are prioritized)
    gae_lambda=0.99,    # factor for trade-off of bias vs variance (< lambda ~= short term rewards are prioritized)
    ent_coef=0.005,     # entropy coefficient for loss calculation (small pos value can encourage exploration)
    verbose=1,          # output
)

Using cuda device


In [23]:
model.learn(
    total_timesteps=1000000,
    progress_bar=True
)

Output()

Streaming output truncated to the last 5000 lines.
|    value_loss           | 1.46e+03    |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 84.4         |
|    ep_rew_mean          | -112         |
| time/                   |              |
|    fps                  | 804          |
|    iterations           | 8            |
|    time_elapsed         | 40           |
|    total_timesteps      | 32768        |
| train/                  |              |
|    approx_kl            | 0.0068435757 |
|    clip_fraction        | 0.111        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.35        |
|    explained_variance   | -6.32e-05    |
|    learning_rate        | 0.0005       |
|    loss                 | 687          |
|    n_updates            | 70           |
|    policy_gradient_loss | -0.00483     |
|    value_loss           | 1.02e+03     |
-----

**Evaluate the Agent**

1. Wrap the environment in a Monitor
2. Check the performance of the agent
3. Upload to Hugging Face

In [24]:
eval_env = Monitor(gym.make('LunarLander-v2'))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=276.39 +/- 12.89769737457947


In [27]:
model_name = 'ppo-LunarLander-v2'
model.save(model_name)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
# authorize using token
notebook_login()
!git config --global credential.helper store

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
from stable_baselines3.common.vec_env import DummyVecEnv


env_id = 'LunarLander-v2'

model_architecture = 'PPO'

repo_id = 'elainechan01/ppo-LunarLander-v2'   # {organization}/{model architecture}-{environment}

commit_message = 'Upload PPO LunarLander-v2 trained agent'

eval_env = DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode='rgb_array'))])

package_to_hub(
    model=model,
    model_name=model_name,
    model_architecture=model_architecture,
    env_id=env_id,
    eval_env=eval_env,
    repo_id=repo_id,
    commit_message=commit_message,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Saving video to /tmp/tmprjde12ly/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmprjde12ly/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmprjde12ly/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmprjde12ly/-step-0-to-step-1000.mp4
ℹ Pushing repo elainechan01/ppo-LunarLander-v2 to the Hugging Face
Hub


...

ppo-LunarLander-v2.zip:   0%|          | 0.00/147k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

policy.pth:   0%|          | 0.00/43.8k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/elainechan01/ppo-LunarLander-v2/tree/main/


CommitInfo(commit_url='https://huggingface.co/elainechan01/ppo-LunarLander-v2/commit/56d277698876db98359de3229d558b7709e785cf', commit_message='Upload PPO LunarLander-v2 trained agent', commit_description='', oid='56d277698876db98359de3229d558b7709e785cf', pr_url=None, pr_revision=None, pr_num=None)

**[Optional] Load a model from the Hub**

In [ ]:
!pip install shimmy

In [ ]:
filename = 'ppo-LunarLander-v2.zip'

custom_objects = {
    'learning_rate': 0.0,
    'lr_schedule': lambda _: 0.0,
    'clip_range': lambda _: 0.0,
}

checkpoint = load_from_hub(repo_id, filename)
hub_model = PPO.load(checkpoint, custom_objects=custom_objects, print_system_info=True)


In [ ]:
# evaluate agent
hub_eval_env = Monitor(gym.make('LunarLander-v2'))
hub_mean_reward, hub_std_reward = evaluate_policy(hub_model, hub_eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")